## RED NEURONAL RECURRENTE - (*RNN - Recurrent Neural Network o Red de ELMAN*)   

Las redes de Elman son el modelo más simple de Red Neuronal Recurrente (RNN). Tienen la misma estructura que las redes neuronales vistas hasta ahora, salvo por una única circunstancia: se permite que cada neurona se **retroalimente a sí misma**.

<img src="./img/Elman.jpg">   

Donde $h_t$ es el estado de la neurona en el momento $t$, $h_{t-1}$ su estado en el momento inmediatamente anterior; $w_i$ representa los pesos sinápticos, $d_i$ los valores de activación de las neuronas de la capa anterior y $b$ el sesgo(NO APARECE EN LA IMAGEN). Como se puede observar, existe un término extra $Uh_{t-1}$ que no existe en el caso de las redes no recurrentes, y que en este caso permite que cada neurona se excite a sí misma.    

Como se puede ver, por tanto, en las neuronas del modelo de Elman se forma un pequeño bucle de retroalimentación, mediante el cual el axón de la neurona excita una de sus propias dendritas. El peso sináptico de esta conexión de retroalimentación es $U$.

El valor de activación en el momento anterior $h_{t-1}$ contribuye a la suma total de excitaciones de la célula $(x)$, a través del coeficiente $U$. Este coeficiente funcionará de forma equivalente a un peso sináptico.

## EJEMPLO DE RNN


Este es un ejemplo sencillo de una RNN que va a permitir llevar a cabo la predicción de caracteres usando un conjunto de datos (cadena de texto).

### IMPORTAR LIBRERÍAS NECESARIAS

In [1]:
import warnings
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam


2025-03-19 21:49:41.284164: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-19 21:49:41.338785: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742420981.409639   45652 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742420981.429440   45652 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-19 21:49:41.502000: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Desactivamos los WARNINGS

In [2]:
warnings.filterwarnings("ignore")

### DEFINICIÓN DEL CONJUNTO DE DATOS Y CREACIÓN DEL VOCABULARIO.

Generamos el conjunto de datos a partir de una frase sencilla y, seguidamente, creamos un vocabulario ordenando el conjunto de carácteres que componen la frase.   


In [16]:
# Definimos el conjunto de datos (secuencia de texto)
text = "hola mundo"
# Crear un vocabulario de caracteres
vocab = sorted(set(text))
vocab_size = len(vocab)

print("Vocabulario: ", vocab)
print("Tamaño del vocabulario: ", vocab_size)

Vocabulario:  [' ', 'a', 'd', 'h', 'l', 'm', 'n', 'o', 'u']
Tamaño del vocabulario:  9



Seguidamente se crea un diccionario mapeando los caracteres del vocabulario con índices para, a continuación, transformar la secuencia de caracteres que conforman la frase en una secuencia de indices.

In [17]:
# Crear un diccionario de mapeo de caracteres a índices
char_to_idx = {char: idx for idx, char in enumerate(vocab)}
idx_to_char = np.array(vocab)
print("Mapeo de caracteres a índices: ", char_to_idx)
print("Mapeo de índices a caracteres: ", idx_to_char)

# Convertir la secuencia de texto a una secuencia de índices
text_as_int = np.array([char_to_idx[c] for c in text])
print("Texto como índices: ", text_as_int)

Mapeo de caracteres a índices:  {' ': 0, 'a': 1, 'd': 2, 'h': 3, 'l': 4, 'm': 5, 'n': 6, 'o': 7, 'u': 8}
Mapeo de índices a caracteres:  [' ' 'a' 'd' 'h' 'l' 'm' 'n' 'o' 'u']
Texto como índices:  [3 7 4 1 0 5 8 6 2 7]


### PREPARACIÓN DE DATOS

Para preparar los datos, en el siguiente bloque de código se establece la longitud de la secuencia de entrada (también denominada "ventana") a un valor de 4 caracteres, por ejemplo: "mund" -> "o".      

En este caso, para el entrenamiento se generan un total de 6 ejemplos para la secuencia de partida: "hola mundo".    

Finalizado este proceso, se dispondrá de los datos de entrenamiento formando un conjunto de pares de secuencia de entrada y salida.

In [19]:
# Preparar los datos de entrenamiento y etiquetas
seq_length = 4 # Tamaño de la ventana = 4. Se toman 4 caracteres como entrada y se predice el siguiente caracter
examples_per_epoch = len(text) - seq_length 

In [20]:
# Crear las secuencias de entrada y salida
inputs = np.array([text_as_int[i:i+seq_length] for i in range(examples_per_epoch)])
targets = np.array([text_as_int[i+seq_length] for i in range(examples_per_epoch)])

# Reshape para cumplir con el formato esperado por la RNN
inputs = np.reshape(inputs, (examples_per_epoch, seq_length, 1)) # se corresponde con (batch_size, seq_length, input_dim)

# Usar tf.data.Dataset para manejar los datos
# y crear un dataset a partir de los tensores de entrada y salida (inputs, targets)
dataset = tf.data.Dataset.from_tensor_slices((inputs, targets)) #from_tensor_slices crea un dataset a partir de tensores, donde cada tensor es una muestra de entrada y salida
# batch(1) indica que se toma un solo ejemplo por batch y drop_remainder=True indica que se descartan los ejemplos que no se ajustan al tamaño del batch
dataset = dataset.batch(1, drop_remainder=True) 


In [21]:
print("Ejemplos por época: ", examples_per_epoch)
print("Entradas: ", inputs.shape)
print("Etiquetas: ", targets.shape)

Ejemplos por época:  6
Entradas:  (6, 4, 1)
Etiquetas:  (6,)


### DISEÑO DE LA ARQUITECTURA DE LA RNN

In [22]:
# Crear el modelo RNN
model = Sequential([
    SimpleRNN(50,                           # Número de unidades en la capa oculta (50)
              input_shape=(seq_length, 1),  # Tamaño de la ventana y número de características (1)
              return_sequences=False),      #return_sequences=False indica que solo se devuelve la salida de la última capa
    
    Dense(vocab_size,                       # Número de unidades en la capa de salida (vocab_size, tamaño del vocabulario)
          activation='softmax')             # Función de activación de la capa de salida (softmax), ya que se trata de un problema de clasificación multiclase.
])



### COMPILACIÓN

In [ ]:
# Compilar el modelo
model.compile(optimizer=Adam(),                         # Optimizador Adam por defecto
              loss='sparse_categorical_crossentropy')   #sparse_categorical_crossentropy ya que se trata de un problema de clasificación multiclase

### ENTRENAMIENTO

In [24]:
# Entrenar el modelo
model.fit(dataset, epochs=100)


Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 2.5336
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 2.1155 
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.8852 
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.7024 
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.5499 
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.4195 
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.3062 
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.2069 
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.1193 
Epoch 10/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.0418 
Epoch 11/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.9730 
Epoch 12/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.9117 
Epoch 13/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.8570 
Epoch 14/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.8081 
Epoch 15/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.7643 
Epoch 16/100
6/6 ━━━

### PREDICCIÓN

In [27]:
# Función para predecir el siguiente carácter en una secuencia dada
def predict_next_char(model, input_text):
    input_eval = np.array([char_to_idx[c] for c in input_text])         # Convertir la entrada a índices
    input_eval = np.reshape(input_eval, (1, len(input_eval), 1))        # Reshape para cumplir con el formato esperado por la RNN
    prediction = model.predict(input_eval)                              # Realizar la predicción
    predicted_idx = np.argmax(prediction)                               # Obtener el índice del carácter predicho
    return idx_to_char[predicted_idx]                                   # Obtener el carácter correspondiente al índice predicho

In [28]:
# Probar el modelo
input_text = "mund"
predicted_char = predict_next_char(model, input_text)
print(f"Entrada: '{input_text}' -> Siguiente carácter predicho: '{predicted_char}'")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step
Entrada: 'mund' -> Siguiente carácter predicho: 'o'


## **NOTA SOBRE LAS RNN Y LAS TAREAS SOBRE SERIES TEMPORALES**   

Las Redes Neuronales Recurrentes (RNN) se utilizan ampliamente para tareas con series temporales, aunque ciertas variantes son más adecuadas que las RNNs básicas.

**¿Por qué las RNNs para series temporales?**

* **Dependencia temporal:**
    * Las series temporales son secuencias de datos donde el orden es crucial. Las RNNs están diseñadas para manejar datos secuenciales, ya que mantienen una "memoria" de las entradas anteriores. Esto les permite capturar dependencias temporales, es decir, cómo los valores pasados influyen en los valores futuros.
    * Por ejemplo, para predecir el precio de una acción, es importante considerar los precios anteriores, ya que suelen existir patrones y tendencias a lo largo del tiempo.
* **Procesamiento de secuencias de longitud variable:**
    * Las RNNs pueden procesar secuencias de longitud variable, lo que es útil para series temporales que pueden tener diferentes duraciones.

**Tipos de RNNs más utilizados para series temporales:**

* **Redes LSTM (Long Short-Term Memory):**
    * Las LSTM son una variante de las RNNs que abordan el problema del "desvanecimiento del gradiente", que dificulta el aprendizaje de dependencias a largo plazo.
    * Las LSTM utilizan "compuertas" que controlan el flujo de información, lo que les permite recordar información relevante durante períodos prolongados.
    * Son muy efectivas para series temporales con patrones complejos y dependencias a largo plazo, como la predicción del precio de acciones o el análisis del lenguaje natural.
* **Redes GRU (Gated Recurrent Units):**
    * Las GRU son una versión simplificada de las LSTM que también abordan el problema del desvanecimiento del gradiente.
    * Tienen menos compuertas que las LSTM, lo que las hace computacionalmente más eficientes.
    * Las GRU suelen ofrecer un rendimiento similar a las LSTM en muchas tareas de series temporales.

**CONCLUSIÓN:**

* Las RNNs, especialmente las LSTM y GRU, son herramientas poderosas para el análisis y la predicción de series temporales debido a su capacidad para capturar dependencias temporales.
* Si bien las RNN basica se pueden usar, LSTM y GRU dan mejor rendimiento con series temporales.
